Isin are consider unique, if several documents have the same isin, merge the results.
Only consider one value for each field only cities can have multiple values. (and 1 isin can have multiple files)
Need to math the name in the files, may not be an exact match with the db or reality because the exact name is not always
well diffined (in the db at least...)


In [1]:
# path
data0='data_0'
data1='data_1'

In [61]:
# import the labels an place them in a dict with key == isin
import csv
import os
import copy
import collections
import re
import numpy as np

class label(dict):
    
    def __init__(self):
        '''
        create an empty dict for holding the label info.
        '''
        self.header=[]
        
    @classmethod
    def from_cvs(cls,datafolder):
        '''
        Create a new dictionary with the all the label loaded for the labels and cities cvs
        file from the folder datafolder.
        
        args:
            datafolder: the folder where the labels.csv and cities.csv files are.
        '''
        assert os.path.isdir(datafolder)
        self=cls()
        l=0
        with open(os.path.join(data0,'labels.csv'),'r') as csvfile:
                reader=csv.reader(csvfile)
                header=reader.next()    
                l=len(header)
                for data in reader:
                    fileid_t=data[0]
                    isin_t=data[1]
                    # safty check:
                    if isin_t not in self:
                        if len(data) != l:
                            print 'icoherent data lengths'
                        self[isin_t]=data[2:]
                        self[isin_t].insert(0,[fileid_t])
                    else:
                        self[isin_t][0].append(fileid_t)
                        if self[isin_t][1:] != data[2:]:
                            print self[isin_t][1:] 
                            print data[2:]
                            raise ValueError('inconsistancy !')         
        l-=1
        with open(os.path.join(data0,'cities.csv'),'r') as csvfile:
                reader=csv.reader(csvfile)
                header2=reader.next()
                header=header[2:]
                header.extend(header2[2:])
                self.header=header
                self.header.insert(0,header2[0])
                print self.header
                for data in reader:      
                    fileid_t=data[0]
                    isin_t=data[1]
                    data=data[2:]

                    # safty check:
                    if fileid_t not in self[isin_t][0]:
                        raise ValueError('file id do not match between cities and lables !')

                    if len(self[isin_t]) == l: # still no cites 
                        self[isin_t].extend([[data[0]],[data[1]]])
                    else: 
                        self[isin_t][-2].append(data[0])
                        self[isin_t][-1].append(data[1])
                        
        for item in self:
            if len(self[item]) < len(self.header):
                self[item].extend([None,None])
                        
        return self
        
    @classmethod
    def from_folder(cls, datafolder, header):
        '''
        Initialize the dictionary with all the files in the given folder. The files are not read,
        this method only import all the files names and extract the isin from the name of
        the file is possible for each of them, and the array
        of data is initialized for each of the isin, all the data are initialized to None (appart for the 
        filepath... ).
        
        if the isin is unknow, the files are append to the list self.unknown 
        
        The files are expected to be formated as (fileid)_i(isin)_(stuff...)
        
        args:
            datafolder: folder where the data are.
            header: header for the data, this is the label for each of the element stored for each isin.
                    the first data is always the 'fileId', so you should always have the label as a fist 
                    element of your header.
                    e.g: header= ['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT', 'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 
                    'SEC.SUB.ID', 'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 'issuerCountry', 
                    'guarantorName', 'guarantorCity', 'guarantorCountry', 'City.Id', 'City.Name']
                    
        '''
        assert os.path.isdir(datafolder)
        self=cls()
        self.header=header
        self.unknown=[]
        for filename in os.listdir(datafolder):
            isin=re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',filename)
            if len(isin) > 1 :
                raise ValueError('multpiple isin !')
            elif len(isin) == 1:
                if isin[0] in self:
                    self[isin[0]][0].append(filename)
                else:
                    self[isin[0]]=[[filename]]
                    self[isin[0]].extend([None]*(len(self.header)-1))
            #isin=filename.split('_')[1]
            #if 'I' == isin[0].upper():
            #    if isin[1:] in self:
            #        self[isin[1:]][0].append(filename)
            #    else:
            #        self[isin[1:]]=[[filename]]
            #        self[isin[1:]].extend([None]*(len(self.header)-1))
            #elif re.match('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', isin):              
            #    if isin in self:
            #        self[isin][0].append(filename)
            #    else:
            #        self[isin]=[[filename]]
            #        self[isin].extend([None]*(len(self.header)-1))
            else:
                self.unknown.append(filename)
        return self
        
    def get_headers(self):
        '''
        Retrun the header as a list, this is all the possible filed store for each isin.
        '''
        return copy.deepcopy(self.header)
        
    def get_isin(self, isin, label=None):
        '''
        Provide the data for a given isin and label, all the possible label are given by get_headers().
            
        args:
            isin: the isin...
            lable: data data you want for this isin, is None, all the data for this isin are returned
        '''
        if label is not None:
            assert label in self.header
            return copy.deepcopy(self[isin][label])
        else:
            return copy.deepcopy(self[isin])
        
    def get_from_field(self,field, value):
        '''
        Return a list of isin that have the given value for the give field.
        
        The research is done by going through the compleate list, this is therfore ineficient.
        
        args:
            field: one of the label in the header where to look for the value
            value: value to look for.
        '''
        result=[]
        for item in self:
            tmp = self[item][self.header.index(field)]
            if tmp is not None and (tmp == value or value in tmp) :
                result.append(item)
        return result
    
    def empty(self):
        '''
        Empty the all the data (set them to None), only keep the isin and the fileId associeted.
        '''
        l=len(self.header)
        for item in self:
            for x in xrange(1,l):
                self[item][x]=None
            
    def evaluate(self, what, ref, printerr=True):
        '''
        Compare the values from this set to a reference set.
        
        Args:
            what: index of the data to compare (see header label)
            ref: another label object to compare with.
            printerr: print not matching entity
        '''
        total=0
        notcomp=0
        err=np.zeros(len(what))
        lookup=dict()
        for i in xrange(0,len(what)):
            lookup[what[i]]=i
            
        for item in ref:
            if item not in self:
                print item+' is not in the set !'
                notcomp+=1
            else:
                total+=1
                for idx in what:
                    if self[item][idx] != ref[item][idx]:
                        if printerr:
                            print item, self[item][idx], ref[item][idx]
                        err[lookup[idx]]+=1
        print 
        print 'total number of set compared: '+str(total)
        print 'total number of missing set: '+str(notcomp)
        print 'total number of errors for each index: '+str(err)
        print 'error %: '+str(err/total*100)
                    

In [62]:
labels=label.from_cvs(data0)

['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT', 'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID', 'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 'issuerCountry', 'guarantorName', 'guarantorCity', 'guarantorCountry', 'City.Id', 'City.Name']


File ids are unique, not isin (multiple file for one isin...)

In [63]:
labels.evaluate([0,1,2,3],labels)


total number of set compared: 1200
total number of missing set: 0
total number of errors for each index: [ 0.  0.  0.  0.]
error %: [ 0.  0.  0.  0.]


In [65]:
data=copy.deepcopy(labels)
data.empty()
data.evaluate([1],labels,printerr=False)


total number of set compared: 1200
total number of missing set: 0
total number of errors for each index: [ 1200.]
error %: [ 100.]


In [6]:
labels.get_from_field('fileId','0900045c82a4ae5c')

['DE000UZ165E9']

In [6]:
labels.get_isin('XS1183229084')

[['0900045c82d3b389'],
 'USD',
 '0.00E+00',
 '1.00E+06',
 'N',
 'USUB',
 'N',
 'USD',
 'CREDIT SUISSE INTERNATIONAL',
 'LONDON',
 'UNITED KINGDOM',
 '',
 '',
 '',
 ['HONG', 'NEW Y'],
 ['HONG KONG', 'NEW YORK']]

In [19]:
# create a dict with all the files to analize, when the isin can be extracted from the name,
# it is directly used for indexing, if not found in the name, the files are added to the unknown list.
data=label.from_folder(os.path.join(data0,'txt'), header=['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT',
                                                       'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID',
                                                       'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 
                                                       'issuerCountry', 'guarantorName', 'guarantorCity', 
                                                       'guarantorCountry', 'City.Id', 'City.Name']
                        )

In [21]:
print data

{}


Use pdf, an try to extract the text, if it fail, converte to image with high dpi and pre process the image with different parameters
and use tesarac ocr to get the text.

Use the isin to detect is the text seems correct, if isin detectrion fail, reject the de ocr. if all doc are rejected, print a error.

In [33]:
import PyPDF2
pdfFileObj = open('data_0/pdf/0900045c82ef7d9b_IXS1205574913_F_PC_N_M.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print pdfReader.numPages
pageObj = pdfReader.getPage(0)
print pageObj.extractText()

18
 Pricing Supplement dated 26 March 2015 Credit Suisse International 3-Year 10-Month USD Notes due January 2019 linked to the Credit Suisse SPEAR Dynamic Asia Index Total Return USD  Series X (the "Securities") issued pursuant to the Structured Products Programme for the issuance of Notes, Certificates and Warrants PART A  CONTRACTUAL TERMS Terms used herein shall be deemed to be defined as such for the purposes of the Programme Memorandum dated 10 July 2013. This document constitutes the Pricing Supplement of the Securities described herein. Copies of the Programme Memorandum may be obtained from the registered office of the Issuer and the offices of the Agents specified herein.  This Pricing Supplement comprises the final terms for the issuance of the Securities.  This Pricing Supplement does not constitute final terms for the purposes of Article 5.4 of Directive 2003/71/EC as amended by Directive 2010/73/EU Directive (the "Prospectus Directive"). The Issuer is not offering the Sec

In [28]:
pageObj = pdfReader.getPage(2)
print pageObj

˘!( (!  ˜!"$#!-˚"'˙˚˚/˚3)˙*'3˚'˚˚'*ˆ*40˙*˚520˙4ˆˆ'&/ˇ:˛!˚5(˙ˆ'7ˇ)˝,!˙+5˙'45(2#˙˙+0ˇ=)˙+5˙'45(2.˙*ˇ)+5˙'45(2(4)ˇ)+-˚53>˙="&)=)˙+0ˇ)˙˙+0ˇ2"-ˇ)+0ˇ(4)ˇ)+0ˇ-˙˙ˇ)+0ˇ:'˚)ˇ)+0ˇˇˇ)+0ˇ
)ˆˆ˚ˆ)*ˆ˛ˆˆ˙+˛ˆˆ˝ˆˆ˙˚˙ˆ)ˆ˚ˆ˚ˆ˛,-../˘ˆ˙ˆ˙)ˆˆ˚ˆˇ˙)ˆˆ˚ˆ˛˘ˆˆˆˆ˚ˆ)ˆ˚0ˆ˛˚1+ˆ2˚ˆ˘ˆ3˚0ˆ˛ˆˆ1ˆ˛/)ˆˆ˚ˆ˘˛ˆ˛ˆˆ˙˛˛˙+))ˆ˚ˆ˙ˆ˛˛/˛˛)ˆ˛˚˛ˆˆ˛/˚ˆ˛˙ˆˆ˙ˆ˝˚˛˚˙ˆ)ˆˆ˚ˆ4ˆ˝˚˛5ˇˇ3)ˆˆˆ˘)*ˆˆˆˆˆ˙)ˆ˚ˆ˛˚ˆ˛ˆ˛+))ˆ˛ˆ˙ˆ˛)ˆˆ˚ˆ˛˚˙ˆ)ˆ˚ˆ˙ˆ˛˛6˚ˆ˛ˆˆ˛ˆ˝˚˛˙˛˝˛)ˆˆ˚ˆ˛)ˆˆˆ*˛ˆ1)˝ˆ//ˆ˙)ˆˆ3˛˙ˆ˛˛)ˆˆ˙ˆ˚)ˆˆ˛˛˛ˆ˙ˆ˛)ˆˆ˚ˆ˙˙˚˛˛)ˆˆˆ˘/)ˆˆ˛ˆ˚˙)ˆ˚ˆ˘ˆ˛ˆ˚ˆˆˆ4˚)ˆ˙ˆ5)ˆˆ˛ˆ˚3˚ˆ˙ˆˆ˛+)˛)˛˙)˛˝˛+ˆ˚0ˆ˛˘˛˚˙ˆ)ˆ˚ˆ˙ˆ˛˘ˆ1+/˚˙˝)ˆ˘˛˛*˙ˆ˙ˆ˛,789ˇ˙,:;<ˇ˛)ˆˆˆ*ˆ˚ˆ˛˙ˆ˛,-;7/˘ˆ˙ˆ˙3


In [11]:
import PyPDF2
import tempfile
import PythonMagick
import Image
from pytesseract import image_to_string


In [27]:
filename='data_0/pdf/0900045c80b013e1_ixs0586130022_f_pc_n_m.pdf'
pdf_im = PyPDF2.PdfFileReader(file(filename, "rb"))
npage = pdf_im.getNumPages()
pdf_im = None
for p in xrange(0,5):
    img = PythonMagick.Image()
    img.density("600")
    img.read(filename+'['+ str(p) +']')
    f=tempfile.NamedTemporaryFile(mode='w+b',suffix='.jpeg')
    img.write(f.name)
    print image_to_string(Image.open(f.name))
    f.close()

Final Terms
Ill BARCLAYS

BARCLAYS BANK PLC

(Incorporated with limited liability in England and Wales)

 

BARCLAYS CAPITAL (CAYMAN) LIMITED

(Incorporated with limited liability in the Cayman Islands)

GLOBAL STRUCTURED SECURITIES PROGRAMME

for the issue of Securities

BARCLAYS BANK PLC
EUR 5,000,000 Inﬂation Linked Notes due 13 April 2021 (the “Notes”)
Series GSN40028

under the Global Structured Securities Programme

Issue Price: 100 per cent. of par

This document constitutes the final terms of the Notes (the “Final Terms”) described herein for the purposes of Article 5.4 of the
Directive 2003/ 71/ EC and is prepared in connection with the Global Structured Securities Programme established by Barclays Bank
PLC (the “Bank”) and Barclays Capital (Cayman) Limited (“BCCL”) and is supplemental to and should be read in conjunction with the
Base Prospectus dated 6 August 2010, as supplemented and amended from time to time, which constitutes a base prospectus (the
“Base Prospectus”) for 

In [6]:
import PythonMagick
img = PythonMagick.Image()
img.density("350")
img.read('data_0/pdf/0900045c80b013e1_ixs0586130022_f_pc_n_m.pdf') # read in at 300 dpi
img.write("test.PNG")

In [31]:
import html2text
with open('data_0/html/0900045c80b013e1_ixs0586130022_f_pc_n_m_ocr.html','r') as f:
    data=f.read()
v=html2text.html2text(data)

In [23]:
data=None
with open(data0+'/txt/0900045c82d7ac42_ocr.txt','r') as f:
    data=f.read()

In [24]:
import re
tmp=data
b=re.findall('[iI][sS][iI][nN].*\n',tmp)
tmp=' '.join(b)
re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)

['XS1061486434']

In [13]:
labels.get_from_field('fileId','0900045c82d7ac42')

['XS1061486434']

In [11]:
import os
import PyPDF2
import tempfile
import PythonMagick
import Image
from pytesseract import image_to_string
import html2text
import StringIO
import re

# HTML files seems to be the best !!!!!
#
def get_text(filename,isin=None):
    '''
    Recover the text from a file. 
    
    args:
        filename: name of the file to read. Format: txt, html, pdf, doc, jpeg or png
        isin: is the isin is know and provided, check if the extracted isin match
              if not, rais an error. Also help for evaluating the quality of ocr.
              
    
    '''
    assert os.path.isfile(filename)
    if isin is None:
        isin=re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',filename)
        if len(isin)==1:
            isin=isin[0]
        else:
            isin=None
    ext = filename.strip().split('.')[-1].lower()
    
    result=None
    if ext == 'txt':
        with open(filename,'r') as f:
            result=f.read().decode('utf8')
        return result
    elif ext == 'html':
        with open(filename,'r') as f:
            data=f.read().decode('utf8')
        return html2text.html2text(data)
    else:
        npage=1
        notok = True
        result = StringIO.StringIO()
        if ext == 'pdf':
            # try to extract the text directly from the pdf
            pdf_im = PyPDF2.PdfFileReader(file(filename, "rb"))
            npage = pdf_im.getNumPages()            
            for p in xrange(0,npage):
                pageObj = pdf_im.getPage(p)
                result.write(pageObj.extractText())
            tmp=result.getvalue()
            result.close() 
            result=tmp.replace('\n',' ')
            # test if if make sense and if we can extract the isin number, if not, this is probalby robish
            b=re.findall('[iI][sS][iI][nN].{0,30}[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
            tmp=' '.join(b)
            isinfound=re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
            if isin is not None:
                if len(isinfound) > 0 and isin in isinfound:
                    notok = False
            else:
                if len(isinfound) > 0:
                    notok = False  
                    
        if notok: # if it is robish, try to make an ocr...
            result = StringIO.StringIO()  
            for p in xrange(0,npage):
                img = PythonMagick.Image()
                img.density("600")
                img.read(filename+'['+ str(p) +']')
                f=tempfile.NamedTemporaryFile(mode='w+b',suffix='.jpeg')
                img.write(f.name)
                result.write(image_to_string(Image.open(f.name)))
                f.close()
            tmp=result.getvalue()
            result.close()
            result=tmp
        
        return result

In [8]:
print get_text('data_0/pdf/0900045c80b013e1_ixs0586130022_f_pc_n_m.pdf',isin=None)

In [10]:
print get_text('data_0/html/0900045c80b013e1_ixs0586130022_f_pc_n_m_ocr.html',isin=None)

In [10]:
print get_text('data_0/pdf/0900045c82ef7d9b_IXS1205574913_F_PC_N_M.pdf',isin=None)

try to extract the isin of all file in html
-------------------

In [9]:
# create a dict with all the files to analize, when the isin can be extracted from the name,
# it is directly used for indexing, if not found in the name, the files are added to the unknown list.
data=label.from_folder(os.path.join(data0,'html'), header=['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT',
                                                       'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID',
                                                       'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 
                                                       'issuerCountry', 'guarantorName', 'guarantorCity', 
                                                       'guarantorCountry', 'City.Id', 'City.Name']
                        )

In [39]:
ok=[]

def add_isin(data,isin):
    '''
    add the isin to data it isin has only one element.
    
    return True if if has been added, false if several or no isin where provided
    '''
    if len(isin) == 1:
        isin=isin.pop()
        data[isin]=[[filename]]
        data[isin].extend([None]*(len(data.header)-1))
        return True
    
    return False
        
for filename in data.unknown:
    if filename != '0900045c82a4ae5c_26022345_Aktienanleihe_19.11.2014_ocr.html': # this one is too long
        tmp=get_text('data_0/html/'+filename)
        tmp=tmp.replace('\n',' ')
        b=re.findall('[iI][sS][iI][nN].{0,30}[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
        tmp=' '.join(b)
        isin=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
        
        if add_isin(data,isin):
            ok.append(filename)        
        else:
            # they are several isin which one is the right one ?
            # if they are in the title, they usualy are the right one:
            with open('data_0/html/'+filename) as p:
                tmp=p.read()
            tmp0=tmp.replace('\n',' ')
            b=re.findall('<h1.*?</h1>',tmp0)
            tmp=' '.join(b)
            isin1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
            if add_isin(data,isin1):
                ok.append(filename)  
            else:
                # if it is in parentesis they should not be taken.
                tmp=get_text('data_0/html/'+filename)
                tmp=tmp.replace('\n',' ')
                b=re.findall('\(.*?\)',tmp)
                tmp=' '.join(b)
                isin1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
                delta=isin.difference(isin1)
                if add_isin(data,delta):
                    ok.append(filename) 
            
for filename in ok:
    data.unknown.remove(filename)
    
    

In [53]:
def get_isin(filename):
    '''
    try to extract the isin from the file
    '''
    tmp=get_text(filename)
    tmp=tmp.replace('\n',' ')
    b=re.findall('[iI][sS][iI][nN].{0,30}[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
    tmp=' '.join(b)
    isin=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
        
    if len(isin) == 1:
        return isin.pop()
    else:
        # they are several isin which one is the right one ?
        # if they are in the title, they usualy are the right one:
        with open(filename) as p:
            tmp=p.read()
        tmp0=tmp.replace('\n',' ')
        b=re.findall('<h1.*?</h1>',tmp0)
        tmp=' '.join(b)
        isinF1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
        if len(isinF1) == 1 :
            return isinF1.pop()
        else:
            # if it is in parentesis they should not be taken.
            tmp=get_text(filename)
            tmp=tmp.replace('\n',' ')
            b=re.findall('\(.*?\)',tmp)
            tmp=' '.join(b)
            isinF1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
            delta=isinF.difference(isinF1)
                
            if len(isinF1) == 1 :
                return isinF1.pop()  
            else:
                return None


for isin in data:
    filenames=data[isin][0]
    for filename in filenames:
        test=get_isin('data_0/html/'+filename)
        if test is None or test.lower() != isin.lower():
                print isin,test,filename

XS1267294996 None 0900045c83c404a2_IXS1267294996_F_PC_N_ocr.html
XS1336577116 None 0900045c83b5181f_IXS1336577116_F_PC_N.html
XS1190853736 None 0900045c8343d40b_SIGNED Amended & Restated Pricing supplement_Autocall_BRC_Shares(207781553_2) (..._ocr.html
XS1267253299 None 0900045c83afbad4_IXS1267253299_F_PC_N_ocr.html
XS1293123862 None 0900045c83b224e4_IXS1293123862_F_PC_N_ocr.html
XS1313393214 None 0900045c83c0f80f_IXS1313393214_F_PC_N_ocr.html
XS1190878139 None 0900045c82ea6f84_IXS1190878139_F_PC_N_ocr.html
XS1190886314 None 0900045c82ee621c_IXS1190886314_F_PC_N_ocr.html
XS1267292784 None 0900045c83c37d6d_IXS1267292784_F_PC_N_ocr.html
xs1190853736 None 0900045c8343d409_amendment notice xs1190853736_ocr.html
XS1190871472 None 0900045c83364a8b_IXS1190871472_F_PC_N_ocr.html
XS1341173976 None 0900045c83bb0f07_IXS1341173976_F_PC_N_ocr.html
XS1293114440 None 0900045c83ad8a5b_IXS1293114440 _F_PC_N_ocr.html
XS1061486517 CH0210483332 0900045c82d7ac43_IXS1061486517_F_PC_N_ocr.html
XS1190860194 N

KeyboardInterrupt: 

In [54]:
def get_isin(filename):
    '''
    try to extract the isin from the file
    '''
    tmp=get_text(filename)
    tmp=tmp.replace('\n',' ')
    b=re.findall('[iI][sS][iI][nN].{0,30}[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
    tmp=' '.join(b)
    print tmp
    isin=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
        
    if len(isin) == 1:
        return isin.pop()
    else:
        # they are several isin which one is the right one ?
        # if they are in the title, they usualy are the right one:
        with open(filename) as p:
            tmp=p.read()
        tmp0=tmp.replace('\n',' ')
        b=re.findall('<h1.*?</h1>',tmp0)
        tmp=' '.join(b)
        isinF1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
        if len(isinF1) == 1 :
            return isinF1.pop()
        else:
            # if it is in parentesis they should not be taken.
            tmp=get_text(filename)
            tmp=tmp.replace('\n',' ')
            b=re.findall('\(.*?\)',tmp)
            tmp=' '.join(b)
            isinF1=set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))
            delta=isinF.difference(isinF1)
                
            if len(isinF1) == 1 :
                return isinF1.pop()  
            else:
                return None

get_isin('data_0/html/'+'0900045c83c404a2_IXS1267294996_F_PC_N_ocr.html')

ISIN  |  XS1267294996 ISIN:**  |  XS1267294996 ISIN: GB0005405286 ISIN: HK0000069689


In [81]:
with open('/home/albert/Bureau/di hack/data_0/html/0900045c8321602e_DB6PNJ_FT legal.v1_ocr.html') as p:
    tmp=p.read()

In [64]:
tmp=get_text('/home/albert/Bureau/di hack/data_0/html/0900045c8321602e_DB6PNJ_FT legal.v1_ocr.html')

In [102]:
re.findall('<h1(!?</h1>).*?[iI][sS][iI][nN].*?[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9].*?</h1>',tmp)

[]

In [8]:
with open('/home/albert/Bureau/di hack/data_0/html/0900045c8321602e_DB6PNJ_FT legal.v1_ocr.html') as p:
    tmp=p.read()
tmp=tmp.replace('\n',' ')
b=re.findall('<h1.*?</h1>',tmp)
print b
tmp=' '.join(b)
set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))

['<h1 style="padding-left: 156pt;text-indent: 0pt;text-align: left;">DEUTSCHE BANK AG LONDON BRANCH</h1>', '<h1 style="padding-top: 10pt;padding-left: 27pt;text-indent: 0pt;text-align: center;">Issue Price<span class="p">: 100 per cent. of the Nominal Amount (Nominal Amount being EUR 1,000 per Security)</span></h1>', '<h1 style="padding-top: 5pt;padding-left: 41pt;text-indent: 0pt;text-align: center;">WKN / ISIN<span class="p">: DB6PNJ / XS0475698931</span></h1>', '<h1 style="padding-left: 5pt;text-indent: 0pt;text-align: justify;">Terms and Conditions (Product Terms)</h1>', '<h1 style="padding-top: 6pt;padding-left: 5pt;text-indent: 0pt;text-align: justify;">Further Information about the Offering of the Securities</h1>', '<h1 style="padding-left: 124pt;text-indent: 0pt;text-align: center;">Terms and Conditions</h1>', '<h1 style="padding-left: 124pt;text-indent: 0pt;text-align: center;">General Definitions applicable to the Securities</h1>', '<h1 style="padding-top: 4pt;padding-left: 1

{'XS0475698931'}

In [23]:
tmp=get_text('data_0/html/0900045c8343d40b_SIGNED Amended & Restated Pricing supplement_Autocall_BRC_Shares(207781553_2) (..._ocr.html')
#print tmp
tmp=tmp.replace('\n',' ')
b=re.findall('[iI][sS][iI][nN].{0,30}[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp)
print b
tmp=' '.join(b)
print set(re.findall('[a-zA-Z][a-zA-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',tmp))

[u'ISIN: US1101221083', u'ISIN: US7170811035', u'ISIN:  |  XS1190853736']
set([u'XS1190853736', u'US1101221083', u'US7170811035'])


In [40]:
data.unknown

['0900045c83aa81ef_206503_Termsheet_AP_GLOBAL__NIKKVELN13084HU.DKO_NKY__2_15.12._ocr.html',
 '0900045c839008e1_205034_Termsheet_AP_GLOBAL__Airbag_AMZN_DIS_NFLX_2_JPY_SMBCN__ocr.html',
 '0900045c839b32c6_205684_Termsheet_AP_GLOBAL__Airbag_AMZN_FB_UA_2_JPY_SMBCN_JAP_ocr.html',
 '0900045c83a9993d_206443_Termsheet_AP_GLOBAL__Airbag_FB_MET_6_SMBCN_JAPAN_FB__2_ocr.html',
 '0900045c839f9b64_205908_Termsheet_AP_GLOBAL__7974KELN10008MZ_7974__2_15.11.27__ocr.html',
 '0900045c83a0ada5_205949_Termsheet_AP_GLOBAL__Airbag_AAPL_AMZN_TSLA_41_JPY_SMBC_ocr.html',
 '0900045c83ad7126_206594_Termsheet_AP_GLOBAL__Airbag_AAPL_AMZN_FB_GOOGL_1_JPY_S_ocr.html',
 '0900045c839cef21_205793_Termsheet_AP_GLOBAL__Airbag_AMZN_FB_NFLX_1_JPY_SMBCN_J_ocr.html',
 '0900045c83ae485b_206627_Termsheet_AP_GLOBAL__6954KVELN10009MSPB.DKO_6954__2_15_ocr.html',
 '0900045c839ceeed_205791_Termsheet_AP_GLOBAL__Airbag_AAPL_AMZN_TWTR_1_JPY_SMBCN_ocr.html',
 '0900045c82a4ae5c_26022345_Aktienanleihe_19.11.2014_ocr.html']